### Reset GG Colab


In [0]:
!kill -9 -1

### **Install necessary lib and perform authorization (twice verification)**

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

### **Kết nối vào Google Drive để lưu hoặc load model (nếu có)**

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

###  Kiểm tra thông tin GPU Cloud, ta thấy là bộ nhớ được cung cấp khoảng 12GB và notebook đã có thể sử dụng Tensorflow.

In [3]:
# Device
import tensorflow as tf
from tensorflow.python.client import device_lib
print('Current Device: ', tf.test.gpu_device_name() + '\n')
device_lib.list_local_devices()

# RAM: 
!cat /proc/meminfo
# CPU: 
!cat /proc/cpuinfo
  
# Tensorflow version
!pip show tensorflow

Current Device:  /device:GPU:0

MemTotal:       13335236 kB
MemFree:          226520 kB
MemAvailable:   11670440 kB
Buffers:          194292 kB
Cached:         11404804 kB
SwapCached:            0 kB
Active:          1672516 kB
Inactive:       10815292 kB
Active(anon):    1036200 kB
Inactive(anon):    84620 kB
Active(file):     636316 kB
Inactive(file): 10730672 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1496 kB
Writeback:             0 kB
AnonPages:        887056 kB
Mapped:           416876 kB
Shmem:            257020 kB
Slab:             441692 kB
SReclaimable:     403060 kB
SUnreclaim:        38632 kB
KernelStack:        3744 kB
PageTables:         7524 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667616 kB
Committed_AS:    2720644 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages

##Bắt đầu chương trình##
Các bước trước, các bạn đã tạo kết nối, chứng thực tài khoản, kết nối GG Drive và test thử con GPU vừa được cấp phát.

Đến bước này, các bạn đã thực sự bắt đầu chương trình của chính mình tạo ra. Khai báo lớp đối tượng DataSet dùng để lấy dữ liệu từ internet về và load vào đối tượng của lớp DataSet. Đoạn source code này các bạn lấy từ file dataset.py và gần như không chỉnh sửa gì cả :D

In [0]:
import numpy as np
import random
import sys
import tensorflow as tf

class DataSet:
    def __init__(self):
        mnist = tf.contrib.learn.datasets.load_dataset("mnist")
        self.train_data = mnist.train.images  # Returns np.array
        self.train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
        self.eval_data = mnist.test.images  # Returns np.array
        self.eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

        self.curr_training_step = 0
        self.curr_test_step = 0

    def get_train_set_size(self):
        return self.train_data.shape[0]

    def get_test_set_size(self):
        return self.eval_data.shape[0]

    def to_one_hot(self, X):
        one_hot = np.zeros((len(X), 10))
        for i in range(len(X)):
            np.put(one_hot[i, :], X[i], 1)

        return one_hot
    
    def next_batch(self, batch_size):
        X_train_bs = self.train_data[self.curr_training_step * batch_size:self.curr_training_step * batch_size + batch_size,:]
        Y_train_bs = self.train_labels[self.curr_training_step * batch_size:self.curr_training_step * batch_size + batch_size]

        self.curr_training_step = self.curr_training_step + 1
        self.curr_training_step = self.curr_training_step if (
            self.curr_training_step * batch_size < self.get_train_set_size()) else 0

        return (X_train_bs, self.to_one_hot(Y_train_bs))

    def next_batch_test(self, batch_size):
        X_test_bs = self.eval_data[self.curr_test_step * batch_size:self.curr_test_step * batch_size + batch_size,:]
        Y_test_bs = self.eval_labels[self.curr_test_step * batch_size:self.curr_test_step * batch_size + batch_size]
        

        self.curr_test_step = self.curr_test_step + 1
        self.curr_test_step = self.curr_test_step if (self.curr_test_step * batch_size < self.get_test_set_size()) else 0

        return (X_test_bs, self.to_one_hot(Y_test_bs))


## Thừa thắng, khai báo tiếp lớp đối tượng LeNet##
Tương tự như ở bước trước, bước này ta có thể copy code từ file train.py vào và gần như không thay đổi dòng mã nguồn nào cả.

Thực ra phần khai báo numpy và tensorflow là hơi thừa, do đã khai báo ở đằng trước. Tuy nhiên, không sao cả. Mình chỉ minh hoạ rằng, dùng Google Colab rất dễ, copy and paste và sau khi đã cấp phát GPU là chạy mà thôi.

**LƯU Ý**: Chỉ có một chỗ duy nhất cần để ý đó là thư mục lưu trữ. Các bạn cần sử lại thành **"drive/#path to the folder in GGDrive#"**. Trong đó, từ 'drive' thể hiện thư mục gốc trên Google Drive của bạn. Phần <paht to the folder in GGDrive> là đường dẫn trên Drive để đến được thư mục muốn làm việc. Ví dụ nếu file .ipynb của bạn đặt ở thư mục GPU-Run, trong thư mục có thư mục model để chứa mô hình sau khi đã huấn luyện thì ta sẽ sửa lại: 
** model_dir = 'drive/GPU-Run/model'**

In [0]:
import numpy as np
import tensorflow as tf

### CHU Y: THAY DUONG DAN DEN THU MUC TREN Google Drive cua ban
model_dir = 'drive/GPU-Run/model'

tf.logging.set_verbosity(tf.logging.INFO)

class LeNet():
  def __init__(self, weights=None, sess=None, log=True):
    self.X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
    self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    self.log = log
    self.sess = sess

    self.conv_layers()
    self.fc_layers()

    self.probs = tf.nn.softmax(self.logits, name='softmax')

    if weights is not None and sess is not None:
      self.load_weights(weights, sess)

  def conv_layers(self):
    self.parameters = []
    images = self.X

    # Layer 1: Conv
    with tf.name_scope('conv1') as scope:
      kernel = tf.Variable(tf.random_normal([3, 3, 1, 32], dtype=tf.float32, stddev=1e-1, 
        name='weights'))
      conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[32], dtype=tf.float32),
        trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      self.conv1 = tf.nn.relu(out, name=scope)
      self.parameters += [kernel, biases]

    # Layer 1: Pooling
    self.pool1 = tf.nn.max_pool(self.conv1,
      ksize=[1,2,2,1],
      strides=[1,2,2,1],
      padding='SAME',
      name='pool1')

    # Layer 2: Conv
    with tf.name_scope('conv2') as scope:
      kernel = tf.Variable(tf.random_normal([3, 3, 32, 64], dtype=tf.float32, stddev=1e-1, name='weights'))
      conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
      biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
        trainable=True, name='biases')
      out = tf.nn.bias_add(conv, biases)
      self.conv2 = tf.nn.relu(out, name=scope)
      self.parameters += [kernel, biases]

    # Layer 2: Pooling
    self.pool2 = tf.nn.max_pool(self.conv2,
      ksize=[1,2,2,1],
      strides=[1,2,2,1],
      padding='SAME',
      name='pool2')

  def fc_layers(self):
    # fc1
    with tf.name_scope('fc1') as scope:
      shape = int(np.prod(self.pool2.get_shape()[1:]))
      fc1w = tf.Variable(tf.random_normal([shape, 128],
        dtype=tf.float32, stddev=1e-1), name='weights')
      fc1b = tf.Variable(tf.constant(1.0, shape=[128], dtype=tf.float32),
        trainable=True, name='biases')
      pool2_flat = tf.reshape(self.pool2, [-1, shape])
      fc1l = tf.nn.bias_add(tf.matmul(pool2_flat, fc1w), fc1b)
      self.fc1 = tf.nn.relu(fc1l)
      self.dropout1 = tf.nn.dropout(self.fc1, keep_prob=self.keep_prob, name='dropout1')
      self.parameters += [fc1w, fc1b]

    # fc2
    with tf.name_scope('fc2') as scope:
      fc2w = tf.Variable(tf.random_normal([128, 10],
        dtype=tf.float32, stddev=1e-1), name='weights')
      fc2b = tf.Variable(tf.constant(1.0, shape=[10], dtype=tf.float32),
        trainable=True, name='biases')
      fc2l = tf.nn.bias_add(tf.matmul(self.dropout1, fc2w), fc2b)
      self.logits = tf.nn.relu(fc2l)
      self.parameters += [fc2w, fc2b]

  def load_weights(weights, sess):
    None

  def train(self, learning_rate, training_epochs, batch_size, keep_prob):
    # Load dataset for training and testing
    self.dataset = DataSet()

    # Define size of output
    self.Y = tf.placeholder(tf.float32, [None, 10], name='Y')
    # Define cost function
    self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
    # Define optimization method
    self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

    # Start logger
    if self.log:
        tf.summary.scalar('cost', self.cost)
        self.merged = tf.summary.merge_all()
        self.train_writer = tf.summary.FileWriter('./log_train', self.sess.graph)

    self.sess.run(tf.global_variables_initializer())
    self.sess.run(tf.local_variables_initializer())

    print('Training...')
    weights = []
    # For each epoch, feed training data and perform updating parameters
    for epoch in range(training_epochs):
        avg_cost = 0
        # Number of batches = size of training set / batch_size
        total_batch = int(self.dataset.get_train_set_size() / batch_size)

        # For each batch 
        for i in range(total_batch + 1):
            # Get next batch to feed to the network
            batch_xs, batch_ys = self.dataset.next_batch(batch_size)
            feed_dict = {
                self.X: batch_xs.reshape([batch_xs.shape[0], 28, 28, 1]),
                self.Y: batch_ys,
                self.keep_prob: keep_prob
            }

            weights, summary, c, _ = self.sess.run([self.parameters, self.merged, self.cost, self.optimizer],
                                                   feed_dict=feed_dict)
            avg_cost += c / total_batch

        if self.log:
            self.train_writer.add_summary(summary, epoch + 1)

        print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

    print('Training finished!')

    saver = tf.train.Saver()
    ### CHU Y: Chi can sua lai duong dan thu muc chua MODEL
    save_path = saver.save(self.sess, model_dir + "/mnist_lenet.ckpt")
    print("Trainned model is saved in file: %s" % save_path)

  def evaluate(self, batch_size, keep_prob):

    self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
    self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

    N = self.dataset.get_test_set_size()
    print('test.size', N);
    correct_sample = 0
    for i in range(0, N, batch_size):
        batch_xs, batch_ys = self.dataset.next_batch_test(batch_size)

        N_batch = batch_xs.shape[0]

        feed_dict = {
            self.X: batch_xs.reshape([N_batch, 28, 28, 1]),
            self.Y: batch_ys,
            self.keep_prob: keep_prob
        }

        correct = self.sess.run(self.accuracy, feed_dict=feed_dict)
        correct_sample += correct * N_batch

    test_accuracy = correct_sample / N

    print("\nAccuracy Evaluates")
    print("-" * 30)
    print('Test Accuracy:', test_accuracy)

##Tiến hành thí nghiệm##

Để bắt đầu khởi tạo và chạy mô hình, ta gọi 5 dòng lệnh sau đây. Và như vậy là ta đã có thể sử dụng K80 (12GB memory) để có thể train mạng LeNet rồi. Thời gian để các bạn train 40 epoch chỉ mất 2 phút.

In [0]:
tf.reset_default_graph()
sess = tf.Session()
lenet = LeNet(sess=sess, weights=None)

lenet.train(learning_rate=0.001, training_epochs=40, batch_size=1000, keep_prob=0.7)
lenet.evaluate(batch_size=1000, keep_prob=0.7)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Training...
Epoch: 02 cost = 0.220026862
Epoch: 03 cost = 0.131963294
Epoch: 04 cost = 0.095869466
Epoch: 05 cost = 0.076893867
Epoch: 06 cost = 0.064144858
Epoch: 07 cost = 0.056597817
Epoch: 08 cost = 0.049744936
Epoch: 09 cost = 0.043758852
Epoch: 10 cost = 0.039856849
Epoch: 11 cost = 0.035404160
Epoch: 12 cost = 0.031734507
Epoch: 13 cost = 0.031378918
Epoch: 14 cost = 0.028549319
Epoch: 15 cost = 0.026436142
Epoch: 16 cost = 0.025040999
Epoch: 17 cost = 0.023261481
Epoch: 18 cost = 0.020847021
Epoch: 19 cost = 0.019771104
Epoch: 20 cost = 0.017252670
Epoch: 21 cost = 0.016569230
Epoch: 22 cost = 0.016363664
Epoch: 23 cost = 0.014983140
Epoch: 24 cost = 0.014590157
Epoch: 25 cost = 0.014470957
Epoch: 26 cost = 0.014388366
Epoch: 27 cost = 0.013914947
Epoch: 28 cost = 0.012147818
Epoch: 29 c